In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

We collect lexical co-occurrence statistics on all words in
the English Wikipedia, using the WikiExtractor tool2 to retrieve
plain text from the April 2015 dump (ca. 2.8B words),
and using simple regular expressions to segment sentences
and words, and remove URLs and punctuation. We perform
no POS tagging, lemmatisation, case normalisation,
or removal of numbers or symbols.

In [2]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
# On English wiki, 10% sample

sf = open(datapath+'/Corpora/wiki/enwiki_20200520/enwiki_20200520_10pc.txt', 'r', encoding='utf-8')

for lines in range(5):
    print(sf.readline())

Autism is a developmental disorder characterized by difficulties with social interaction and communication, and by restricted and repetitive behavior. 

 Parents often notice signs during the first three years of their child's life. 

 These signs often develop gradually, though some children with autism experience worsening in their communication and social skills after reaching developmental milestones at a normal pace. 

 Autism is associated with a combination of genetic and environmental factors. 

 Risk factors during pregnancy include certain infections, such as rubella, toxins including valproic acid, alcohol, cocaine, pesticides, lead, and air pollution, fetal growth restriction, and autoimmune diseases. 



In [4]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

In [5]:
w10p = PlaintextCorpusReader(datapath+'/Corpora/wiki/enwiki_20200520/','enwiki_20200520_10pc.txt',
                            word_tokenizer = WhitespaceTokenizer()
                            )

In [6]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

We collect word frequency information with the
SRILM language modelling toolkit (Stolcke, 2002), counting
n-grams (n <= 3), treating MWEs as contiguous bigrams
and trigrams), and identify MWE candidates by computing
the Poisson collocation measure (Quasthoff and Wolff,
2002) for all bigrams and trigrams (ca. 23M n-grams).
This method should be readily extensible to include longer
n-grams.

In [7]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [8]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/enwiki_20200520/10pc_stop.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [9]:
stop

{'0',
 '1',
 '2',
 '3',
 'A',
 'He',
 'In',
 'It',
 'New',
 'The',
 'a',
 'also',
 'an',
 'and',
 'are',
 'as',
 'at',
 'be',
 'been',
 'but',
 'by',
 'first',
 'for',
 'from',
 'had',
 'has',
 'have',
 'he',
 'her',
 'his',
 'in',
 'is',
 'it',
 'its',
 'not',
 'of',
 'on',
 'one',
 'or',
 'that',
 'the',
 'their',
 'this',
 'to',
 'two',
 'was',
 'were',
 'which',
 'who',
 'with'}

In [10]:
min_freq = 20
eval_count = 500000

In [11]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/enwiki_20200520/10pc_ngram_eval.pkl')

ngram_eval

,ngram,poisson,len,batch
0,"(of, the)",3.874652e+06,2,-2
1,"(References, External, links)",2.566994e+06,3,1
2,"(External, links)",2.229096e+06,2,2
3,"(in, the)",2.094387e+06,2,-2
4,"(0, 0, 0)",1.798530e+06,3,-2
...,...,...,...,...
499995,"(Work, started, on)",3.883693e+02,3,8
499996,"(of, these, techniques)",3.883679e+02,3,8
499997,"(Stadio, Flaminio)",3.883678e+02,2,1
499998,"(IRE, 3, C)",3.883678e+02,3,1


In [12]:
batch_count = max(ngram_eval.batch)

In [13]:
ngram_eval.batch.value_counts()

 1     94011
 2     79538
 3     69221
 4     56154
 5     48934
 6     38288
 7     30374
-1     29076
 8     22495
 9     19272
 10    11300
-2      1337
Name: batch, dtype: int64

In [14]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [15]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Build model
    print('Building word2vec model')
    sents_mwe = Sent_Seq(w10p, batch_token_mwe)
    
    print(' Training model')
    batch_model = Word2Vec(sents_mwe,
                             min_count = 20,  # 20 matches R&E on EN Wiki
                             size = 400,
                             workers = 8,
                             window = 5,
                             sg = 0,         # CBOW
                             sample = 10e-5, # Subsampling
                             negative = 10
                            )

    # Save model
    print(' Saving model')
    batch_model.save(datapath+'/Models/1 w2v/wiki10pc_batch{}.model'.format(bb+1))
    # Reload looks like    new_model = gensim.models.Word2Vec.load('/tmp/mymodel')


Processing batch 1 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 2 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 3 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 4 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 5 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 6 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 7 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 8 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 9 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 10 of 10
Building word2vec model
 Training model
 Saving model
Wall time: 2d 11h 5min 8s
